In [2]:
import numpy as np
from astropy.coordinates import CartesianRepresentation, ITRS, GCRS
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.utils import iers
import erfa
iers.conf.auto_download = False  # 禁用自动下载

In [2]:
def keplerian_to_cartesian(a, e, i, raan, argp, nu, mu):
    """
    将开普勒轨道根数转换为笛卡尔位置和速度矢量 (IJK坐标系)。

    参数:
    a : float
        半长轴 (m)
    e : float
        偏心率
    i : float
        轨道倾角 (rad)
    raan : float
        升交点赤经 (rad)
    argp : float
        近地点幅角 (rad)
    nu : float
        真近点角 (rad)
    mu : float
        中心天体重力参数 (m^3/s^2)

    返回:
    r_ijk : numpy array
        位置矢量 [X, Y, Z] (m)
    v_ijk : numpy array
        速度矢量 [Vx, Vy, Vz] (m/s)
    """

    # 1. 计算比角动量 h 和瞬时距离 r
    h = np.sqrt(mu * a * (1 - e**2))
    r = (h**2 / mu) / (1 + e * np.cos(nu))

    # 2. 在PQW轨道平面坐标系中的位置和速度
    r_pqw = np.array([r * np.cos(nu), r * np.sin(nu), 0])
    v_pqw = np.array([-(mu/h) * np.sin(nu), (mu/h) * (e + np.cos(nu)), 0])

    # 3. 构建旋转矩阵 (从PQW到IJK)
    # 注意：代码中使用的是旋转矩阵的转置
    R11 = np.cos(raan)*np.cos(argp) - np.sin(raan)*np.sin(argp)*np.cos(i)
    R12 = -np.cos(raan)*np.sin(argp) - np.sin(raan)*np.cos(argp)*np.cos(i)
    R13 = np.sin(raan)*np.sin(i)
    
    R21 = np.sin(raan)*np.cos(argp) + np.cos(raan)*np.sin(argp)*np.cos(i)
    R22 = -np.sin(raan)*np.sin(argp) + np.cos(raan)*np.cos(argp)*np.cos(i)
    R23 = -np.cos(raan)*np.sin(i)
    
    R31 = np.sin(argp)*np.sin(i)
    R32 = np.cos(argp)*np.sin(i)
    R33 = np.cos(i)
    
    rot_matrix = np.array([[R11, R12, R13],
                           [R21, R22, R23],
                           [R31, R32, R33]])

    # 4. 应用旋转，得到IJK坐标系中的矢量
    r_ijk = rot_matrix @ r_pqw
    v_ijk = rot_matrix @ v_pqw

    return r_ijk, v_ijk

if __name__ == "__main__":
    # 定义地球重力参数 (m^3/s^2)
    mu_earth = 3.986004418e14

    # 输入一组开普勒根数 (示例为一个近地轨道)
    a = 6778000.0  # m (约400km高度)
    e = 0.001
    i = np.deg2rad(51.6)  # 转换为弧度
    raan = np.deg2rad(80.0)
    argp = np.deg2rad(40.0)
    nu = np.deg2rad(30.0)

    # 执行转换
    r_vec, v_vec = keplerian_to_cartesian(a, e, i, raan, argp, nu, mu_earth)

    # 打印结果
    print(f"位置矢量 [X, Y, Z]: {r_vec} 米")
    print(f"速度矢量 [Vx, Vy, Vz]: {v_vec} 米/秒")

位置矢量 [X, Y, Z]: [-3490553.25095484  2967414.22232443  4987205.00104197] 米
速度矢量 [Vx, Vy, Vz]: [-2860.20056896 -6818.00547155  2060.09789316] 米/秒


In [7]:
def eci_to_ecef(x_eci, y_eci, z_eci, utc_time):
    # 计算GAST（Astropy内置方法）
    t = Time(utc_time, scale='utc')
    gast = t.sidereal_time('apparent', 'greenwich').radian
    
    # 构建旋转矩阵
    Rz = np.array([
        [np.cos(gast), np.sin(gast), 0],
        [-np.sin(gast), np.cos(gast), 0],
        [0, 0, 1]
    ])
    
    # 应用旋转
    ecef = Rz @ np.array([x_eci, y_eci, z_eci])
    return ecef

if __name__ == "__main__":
    x_eci, y_eci, z_eci = 1000000, 2000000, 3000000  # 假设ECI坐标
    ecef = eci_to_ecef(x_eci, y_eci, z_eci, '2000-01-01 12:00:00')
    print("ECEF坐标 (m):", ecef)

ECEF坐标 (m): [-1785248.68548046  1346434.97094746  3000000.        ]


In [ ]:
def eci_to_ecef_acc(x_eci, y_eci, z_eci, utc_time):
    # 创建 GCRS 坐标（J2000.0 历元）
    coord = SkyCoord(
        x=x_eci, y=y_eci, z=z_eci,
        frame='gcrs',
        representation_type='cartesian',
        obstime=utc_time
    )
    
    # 转换为 ITRS
    coord_trans = coord.transform_to('itrs')
    
    return coord_trans

if __name__ == "__main__":
    t = Time("2000-01-01 12:00:00", scale="utc")
    x_eci, y_eci, z_eci = 1000000, 2000000, 3000000  # 假设ECI坐标
    ecef = eci_to_ecef_acc(x_eci, y_eci, z_eci, '2000-01-01 12:00:00')
    print("ECEF 坐标 (m):", ecef.cartesian)

ECEF 坐标 (m): (-1785232.64476908, 1346634.72812936, 2999919.88443804) 


In [5]:
iers_a = iers.IERS_Auto.open()

# 定义时间
t = Time("2000-01-01 12:00:00", scale='utc')

# TT 两部分儒略日
tt1, tt2 = t.tt.jd1, t.tt.jd2
# UT1 两部分儒略日
ut1_1, ut1_2 = t.ut1.jd1, t.ut1.jd2

# 章动+岁差矩阵
x, y, s = erfa.xys06a(tt1, tt2)
pn = erfa.c2ixys(x, y, s)

# 地球自转角
era = erfa.era00(ut1_1, ut1_2)
# era = erfa.gst00b(ut1_1, ut1_2)

R = np.array([
    [ np.cos(era), np.sin(era), 0.0],
    [-np.sin(era), np.cos(era), 0.0],
    [ 0.0,          0.0,        1.0]
])

# 极移矩阵
# 获取极移参数
x_p, y_p = iers_a.pm_xy(t.jd1, t.jd2)
print(f"x_p = {x_p.to('arcsec')}, y_p = {y_p.to('arcsec')}")

# 小角度近似
xp = x_p.to(u.rad).value
yp = y_p.to(u.rad).value

# s' 修正
sp = erfa.sp00(tt1, tt2)
pom = erfa.pom00(xp, yp, sp)

# 合成总矩阵
total = pom @ R @ pn

print("PN 矩阵:\n", pn)
print("R 矩阵:\n", R)
print("POM 矩阵:\n", pom)
print("总矩阵 (ECI→ECEF):\n", total)

vec_gcrs = np.array([1000e3, 2000e3, 3000e3])  # 7000 km
vec_itrs_via_matrix = total @ vec_gcrs
print("\nExample vector (GCRS) -> ITRS via composed matrix:\n", vec_itrs_via_matrix)
# [-1785232.6434222   1346634.72991934  2999919.88443606]

x_p = 0.0433815 arcsec, y_p = 0.3778705 arcsec
PN 矩阵:
 [[ 1.00000000e+00  9.75665826e-09  2.69461723e-05]
 [-1.05112802e-08  1.00000000e+00  2.80047948e-05]
 [-2.69461720e-05 -2.80047951e-05  9.99999999e-01]]
R 矩阵:
 [[ 0.18158511 -0.98337523  0.        ]
 [ 0.98337523  0.18158511  0.        ]
 [ 0.          0.          1.        ]]
POM 矩阵:
 [[ 1.00000000e+00 -4.63442157e-18  2.10319447e-07]
 [ 3.85303106e-13  1.00000000e+00 -1.83196788e-06]
 [-2.10319447e-07  1.83196788e-06  1.00000000e+00]]
总矩阵 (ECI→ECEF):
 [[ 1.81585123e-01 -9.83375230e-01 -2.24358785e-05]
 [ 9.83375230e-01  1.81585122e-01  2.97514843e-05]
 [-2.51828510e-05 -2.74653141e-05  9.99999999e-01]]

Example vector (GCRS) -> ITRS via composed matrix:
 [-1785232.64476908  1346634.72812936  2999919.88443804]


In [187]:
# cirs = pn @ vec_gcrs
cirs = np.array([1000080.85766708, 2000084.00308899, 2999917.04197223])
tirs = R @ cirs
itrs = pom @ tirs
cirs,tirs,itrs

(array([1000080.85766708, 2000084.00308899, 2999917.04197223]),
 array([-1785233.27571001,  1346640.22388398,  2999917.04197223]),
 array([-1785232.64476908,  1346634.72812937,  2999919.88443804]))

In [ ]:
pn = erfa.pnm06a(t.tt.jd1, t.tt.jd2)
# p = erfa.pmat06(t.tt.jd1, t.tt.jd2)
# n = erfa.num06a(t.tt.jd1, t.tt.jd2)
# b = erfa.bp06(t.tt.jd1, t.tt.jd2)
pn @ vec_gcrs

(array([1000204.64020741, 2000022.10462589, 2999917.04197223]),
 array([1000204.64020741, 2000022.10462589, 2999917.04197223]))

In [208]:
c = SkyCoord(
    x=1000e3*u.m, y=2000e3*u.m, z=3000e3*u.m,
    frame='gcrs',
    representation_type='cartesian',
    obstime=t
)

c.transform_to("cirs").cartesian, c.transform_to("itrs").cartesian

(<CartesianRepresentation (x, y, z) in m
     (1000080.85766708, 2000084.00308899, 2999917.04197223)>,
 <CartesianRepresentation (x, y, z) in m
     (-1785232.64476908, 1346634.72812936, 2999919.88443804)>)

In [210]:
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import CartesianRepresentation, GCRS, CIRS

# 时间（与你之前一致）
t = Time("2000-01-01 12:00:00", scale='utc')

# 1) erfa 的 pn
pn = erfa.pnm06a(t.tt.jd1, t.tt.jd2)

# 2) 用 Astropy 把基向量从 GCRS -> CIRS，构造 Astropy 的变换矩阵
e1 = CartesianRepresentation([1.0, 0.0, 0.0] * u.m)
e2 = CartesianRepresentation([0.0, 1.0, 0.0] * u.m)
e3 = CartesianRepresentation([0.0, 0.0, 1.0] * u.m)

def gcrs_to_cirs_vec(rep):
    g = GCRS(rep, obstime=t)
    cirs = g.transform_to(CIRS(obstime=t))
    return cirs.cartesian.xyz.to(u.m).value  # 返回 numpy array

v1 = gcrs_to_cirs_vec(e1)
v2 = gcrs_to_cirs_vec(e2)
v3 = gcrs_to_cirs_vec(e3)

M_astropy = np.column_stack([v1, v2, v3])

# 输出比较
print("pn (erfa.pnm06a):\n", pn)
print("\nM_astropy (from SkyCoord transforms):\n", M_astropy)
print("\nDiff (M_astropy - pn):\n", (M_astropy - pn))
print("\nMax abs diff:", np.max(np.abs(M_astropy - pn)))


pn (erfa.pnm06a):
 [[ 9.99999998e-01  6.18993850e-05  2.69479054e-05]
 [-6.19001396e-05  9.99999998e-01  2.80031271e-05]
 [-2.69461720e-05 -2.80047951e-05  9.99999999e-01]]

M_astropy (from SkyCoord transforms):
 [[ 1.00000000e+00  9.75665826e-09  2.69461723e-05]
 [-1.05112802e-08  1.00000000e+00  2.80047948e-05]
 [-2.69461720e-05 -2.80047951e-05  9.99999999e-01]]

Diff (M_astropy - pn):
 [[ 1.91581384e-09 -6.18896283e-05 -1.73315474e-09]
 [ 6.18896283e-05  1.91576688e-09  1.66774227e-09]
 [-3.38813179e-21 -3.38813179e-21  0.00000000e+00]]

Max abs diff: 6.188962834338667e-05


In [59]:
# 创建时间对象
t = Time("2000-01-01 12:00:00", scale='utc')
ut1_1, ut1_2 = t.ut1.jd1, t.ut1.jd2

# 计算ERA（不考虑章动）
era = erfa.era00(ut1_1, ut1_2)
print(f"地球自转角 (ERA): {era:.10f} rad")

# 计算包含章动的恒星时
gst_apparent = t.sidereal_time('apparent', 'greenwich')
gst_mean = t.sidereal_time('mean', 'greenwich')
print(f"视-平恒星时差: {(gst_apparent-gst_mean).to(u.arcsec):.4f}")
print(f"视恒星时: {(gst_apparent).to(u.arcsec):.4f}")
print(f"平恒星时: {(gst_mean).to(u.arcsec):.4f}")

gast1 = gst_mean.radian
Rz_mean = np.array([
    [np.cos(gast1), np.sin(gast1), 0],
    [-np.sin(gast1), np.cos(gast1), 0],
    [0, 0, 1]
])

gast2 = gst_apparent.radian
Rz_apparent = np.array([
    [np.cos(gast2), np.sin(gast2), 0],
    [-np.sin(gast2), np.cos(gast2), 0],
    [0, 0, 1]
])

Rz_apparent @ vec_gcrs, Rz_mean @ vec_gcrs

地球自转角 (ERA): 4.8949871025 rad
视-平恒星时差: -12.7803 arcsec
视恒星时: 1009650.8006 arcsec
平恒星时: 1009663.5809 arcsec


(array([-1785248.68548046,  1346434.97094746,  3000000.        ]),
 array([-1785165.25639046,  1346545.58310381,  3000000.        ]))

In [ ]:
# 1. 计算章动/岁差矩阵
pn = erfa.pnm06a(tt1, tt2)

# 2. 计算ERA矩阵
era = erfa.era00(ut1_1, ut1_2)
R = np.array([
    [ np.cos(era), np.sin(era), 0],
    [-np.sin(era), np.cos(era), 0],
    [ 0,           0,           1]
])

# 3. 组合得到包含章动的总旋转
total_rotation = R @ pn  # CIRS → TIRS
total_rotation @ vec_gcrs

array([-1785149.929228  ,  1346750.70872892,  2999917.04197223])

In [ ]:
# ECEF 坐标 (m): [-1785232.64476908  1346634.72812936  2999919.88443804] m

In [ ]:
erfa.gst06a(t.ut1.jd1, t.ut1.jd2, t.tt.jd1, t.tt.jd2)

np.float64(4.89492521286939)

In [ ]:
era = erfa.era00(ut1_1, ut1_2)
era

np.float64(4.894987102497796)

In [ ]:
import erfa
from astropy.time import Time
import numpy as np

def era_to_gast(era, tt1, tt2):
    """将地球自转角(ERA)转换为格林尼治视恒星时(GAST)"""
    # 计算岁差修正项 (IAU 2006)
    # 从J2000.0起算的儒略世纪数
    t = ((tt1 - 2451545.0) + tt2) / 36525.0
    
    # 多项式系数 (IAU 2006, 单位: radians)
    poly = np.array([-0.0000000368, -0.000029956, -0.00000044, 
                     1.3915817, 4612.156534, 0.014506]) / 86400.0
    
    # 计算平恒星时修正
    gmst = era + np.polyval(poly, t)
    
    # 计算章动修正 (二分点方程)
    # 获取章动参数
    dpsi, deps = erfa.nut06a(tt1, tt2)
    # 计算平均黄赤交角
    eps = erfa.obl06(tt1, tt2)
    # 二分点方程 = Δψ * cos(ε)
    ee = dpsi * np.cos(eps)
    
    # 总视恒星时
    gast = gmst + ee
    
    # 规范化到[0, 2π]
    gast = gast % (2 * np.pi)
    
    return gast

t = Time('2024-03-20 12:00:00', scale='utc')
era = erfa.era00(t.ut1.jd1, t.ut1.jd2)
gast_calculated = era_to_gast(era, t.tt.jd1, t.tt.jd2)
gast_direct = erfa.gst06a(t.ut1.jd1, t.ut1.jd2, t.tt.jd1, t.tt.jd2)

print(f"计算值: {gast_calculated:.12f} rad")
print(f"直接值: {gast_direct:.12f} rad")
print(f"差值: {(gast_calculated - gast_direct):.3e} rad")

计算值: 6.264700145098 rad
直接值: 6.257187348101 rad
差值: 7.513e-03 rad


In [ ]:
def gast_to_era(gast, tt1, tt2):
    """将格林尼治视恒星时(GAST)转换为地球自转角(ERA)"""
    # 计算章动修正
    dpsi, deps = erfa.nut06a(tt1, tt2)
    eps = erfa.obl06(tt1, tt2)
    ee = dpsi * np.cos(eps)
    
    # 计算平恒星时
    gmst = gast - ee
    
    # 计算岁差修正
    t = ((tt1 - 2451545.0) + tt2) / 36525.0
    poly = np.array([-0.0000000368, -0.000029956, -0.00000044, 
                     1.3915817, 4612.156534, 0.014506]) / 86400.0
    poly_corr = np.polyval(poly, t)
    
    # 计算ERA
    era = gmst - poly_corr
    
    # 规范化到[0, 2π]
    era = era % (2 * np.pi)
    
    return era

t = Time('2024-03-20 12:00:00', scale='utc')
gast = erfa.gst06a(t.ut1.jd1, t.ut1.jd2, t.tt.jd1, t.tt.jd2)
era_calculated = gast_to_era(gast, t.tt.jd1, t.tt.jd2)
era_direct = erfa.era00(t.ut1.jd1, t.ut1.jd2)

print(f"计算值: {era_calculated:.12f} rad")
print(f"直接值: {era_direct:.12f} rad")
print(f"差值: {(era_calculated - era_direct):.3e} rad")

计算值: 6.244278709249 rad
直接值: 6.251791506246 rad
差值: -7.513e-03 rad


In [ ]:
from astropy.constants import G, M_earth
mu_earth = (G * M_earth).to(u.km**3 / u.s**2).value  # 地球引力常数

def kepler_propagation(a, e, i, raan, argp, M0, t0, t):
    """
    简单二体开普勒解析传播
    a,e,i,raan,argp,M0 单位分别: km, -, rad, rad, rad, rad
    t0, t: astropy Time
    返回 r,v (km, km/s)
    """
    n = np.sqrt(mu_earth / a**3)  # 平均运动
    M = M0 + n * (t - t0).to(u.s).value  # 平近点角

    # 解开普勒方程 M = E - e*sinE
    def solve_kepler(M, e):
        E = M
        for _ in range(50):
            E = E - (E - e*np.sin(E) - M)/(1 - e*np.cos(E))
        return E
    E = solve_kepler(M, e)
    # 真近点角
    f = 2*np.arctan2(np.sqrt(1+e)*np.sin(E/2),
                     np.sqrt(1-e)*np.cos(E/2))
    r_norm = a*(1 - e*np.cos(E))
    # 在轨道平面坐标系
    r_pf = np.array([r_norm*np.cos(f), r_norm*np.sin(f), 0])
    v_pf = np.sqrt(mu_earth*a)/r_norm * np.array([-np.sin(E), np.sqrt(1-e**2)*np.cos(E), 0])
    # 转换矩阵 (PQW -> ECI)
    cosO, sinO = np.cos(raan), np.sin(raan)
    cosi, sini = np.cos(i), np.sin(i)
    cosw, sinw = np.cos(argp), np.sin(argp)
    R = np.array([
        [cosO*cosw - sinO*sinw*cosi, -cosO*sinw - sinO*cosw*cosi, sinO*sini],
        [sinO*cosw + cosO*sinw*cosi, -sinO*sinw + cosO*cosw*cosi, -cosO*sini],
        [sinw*sini, cosw*sini, cosi]
    ])
    r = R @ r_pf
    v = R @ v_pf
    return r, v


In [30]:
dt_targets = [5, 1, 3]
order = np.argsort(dt_targets)       # [1, 2, 0]
order.argsort()                      # [2, 0, 1]

print("dt_targets 排序:", np.array(dt_targets)[order][order.argsort()])   # [1, 3, 5]
order, order.argsort()  

dt_targets 排序: [5 1 3]


(array([1, 2, 0]), array([2, 0, 1]))

In [3]:
from astropy import units as u
from astropy.time import Time
from poliastro.bodies import Earth
from poliastro.twobody import Orbit

# 定义轨道
epoch = Time("2000-09-18T00:00:00", scale="utc")
orb = Orbit.from_classical(
    attractor=Earth,
    a=7000e3*u.m, ecc=0.001*u.one,
    inc=51.6*u.deg, raan=247.46*u.deg, argp=130.536*u.deg, nu=10*u.deg,
    epoch=epoch
)

new_orb = orb.propagate(1 * u.day)
print(new_orb.r.to(u.m), new_orb.v.to(u.m/u.s))

[ 3303997.39098724 -3082265.0596157   5340882.90406655] m [3795.15827016 6390.07757015 1332.03320852] m / s


In [79]:
import numpy as np
from astropy import units as u
from astropy.time import Time
from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.twobody.propagation import CowellPropagator
from poliastro.core.perturbations import J2_perturbation
from poliastro.core.propagation import func_twobody

# 初始轨道
epoch = Time("2025-09-18T00:00:00", scale="utc")
orb = Orbit.from_classical(
    attractor=Earth,
    a=7000e3*u.m, ecc=0.001*u.one,
    inc=51.6*u.deg, raan=247.46*u.deg, argp=130.536*u.deg, nu=10*u.deg,
    epoch=epoch
)
# J2 常数
J2 = Earth.J2.value
R = Earth.R.to(u.km).value
mu = Earth.k.to(u.km**3 / u.s**2).value

def f(t0, u_, k):
    du_kep = func_twobody(t0, u_, k)
    ax, ay, az = J2_perturbation(
        t0, u_, k, J2=Earth.J2.value, R=Earth.R.to(u.km).value
    )
    du_ad = np.array([0, 0, 0, ax, ay, az])
    return du_kep + du_ad

tof = 1 * u.day
new_orb = orb.propagate(tof, method=CowellPropagator())
# new_orb = orb.propagate(tof, method=CowellPropagator(f=f))

print("位置 (km):", new_orb.r.to(u.m).value)
print("速度 (km/s):", new_orb.v.to(u.m/u.s).value)

位置 (km): [ 3303997.39075364 -3082265.06000574  5340882.90398298]
速度 (km/s): [3795.15827039 6390.07756993 1332.03320888]


In [80]:
new_orb1 = orb.propagate(tof)

print("位置 (km):", new_orb1.r.to(u.m).value)
print("速度 (km/s):", new_orb1.v.to(u.m/u.s).value)

位置 (km): [ 3303997.39098724 -3082265.0596157   5340882.90406655]
速度 (km/s): [3795.15827016 6390.07757015 1332.03320852]


In [78]:
r0v0 = orb._state.to_vectors()
r0 = r0v0.r.to_value(u.m)
v0 = r0v0.v.to_value(u.m/u.s)
r0, v0

(array([4619494.36812227, 3928124.34736963, 3483338.09842163]),
 array([-1504.31074401,  5823.44333568, -4569.43615947]))